In [ ]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "HDLLDL_srWGS_dataset" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_55136786_person_sql <- paste("
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `person` person 
    LEFT JOIN
        `concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `cb_search_person` p 
            WHERE
                has_whole_genome_variant = 1 ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `cb_criteria` cr       
                        WHERE
                            concept_id IN (40782589, 40795800)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
person_55136786_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "person_55136786",
  "person_55136786_*.csv")
message(str_glue('The data will be written to {person_55136786_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_55136786_person_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  person_55136786_path,
  destination_format = "CSV")



In [ ]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {person_55136786_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(gender = col_character(), race = col_character(), ethnicity = col_character(), sex_at_birth = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_55136786_person_df <- read_bq_export_from_workspace_bucket(person_55136786_path)

dim(dataset_55136786_person_df)

head(dataset_55136786_person_df, 5)

In [ ]:
summary(dataset_55136786_person_df)

In [ ]:
table(duplicated(dataset_55136786_person_df$person_id))

In [ ]:
dataset_55136786_person_df$sexM = NA
dataset_55136786_person_df$sexM[dataset_55136786_person_df$sex_at_birth == "Female"] = 0
dataset_55136786_person_df$sexM[dataset_55136786_person_df$sex_at_birth == "Male"] = 1

In [ ]:
write.table(
    dataset_55136786_person_df[, c("person_id", "sexM", "date_of_birth")],
    sep = "\t",
    na = "NA",
    row.names = FALSE,
    quote = FALSE,
    file = "HDLLDL_srWGS_person.txt")

In [ ]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "HDLLDL_srWGS_dataset" for domain "measurement" and was generated for All of Us Controlled Tier Dataset v7
dataset_55136786_measurement_sql <- paste("
    SELECT
        measurement.person_id,
        measurement.measurement_concept_id,
        m_standard_concept.concept_name as standard_concept_name,
        m_standard_concept.concept_code as standard_concept_code,
        m_standard_concept.vocabulary_id as standard_vocabulary,
        measurement.measurement_datetime,
        measurement.measurement_type_concept_id,
        m_type.concept_name as measurement_type_concept_name,
        measurement.operator_concept_id,
        m_operator.concept_name as operator_concept_name,
        measurement.value_as_number,
        measurement.value_as_concept_id,
        m_value.concept_name as value_as_concept_name,
        measurement.unit_concept_id,
        m_unit.concept_name as unit_concept_name,
        measurement.range_low,
        measurement.range_high,
        measurement.visit_occurrence_id,
        m_visit.concept_name as visit_occurrence_concept_name,
        measurement.measurement_source_value,
        measurement.measurement_source_concept_id,
        m_source_concept.concept_name as source_concept_name,
        m_source_concept.concept_code as source_concept_code,
        m_source_concept.vocabulary_id as source_vocabulary,
        measurement.unit_source_value,
        measurement.value_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `measurement` measurement 
        WHERE
            (
                measurement_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (3038553, 40782589, 40795800)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                measurement.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `cb_criteria` cr       
                                WHERE
                                    concept_id IN (40782589, 40795800)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 1 
                                AND is_selectable = 1) 
                            AND is_standard = 1 )) criteria ) )
            )) measurement 
    LEFT JOIN
        `concept` m_standard_concept 
            ON measurement.measurement_concept_id = m_standard_concept.concept_id 
    LEFT JOIN
        `concept` m_type 
            ON measurement.measurement_type_concept_id = m_type.concept_id 
    LEFT JOIN
        `concept` m_operator 
            ON measurement.operator_concept_id = m_operator.concept_id 
    LEFT JOIN
        `concept` m_value 
            ON measurement.value_as_concept_id = m_value.concept_id 
    LEFT JOIN
        `concept` m_unit 
            ON measurement.unit_concept_id = m_unit.concept_id 
    LEFT JOIn
        `visit_occurrence` v 
            ON measurement.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` m_visit 
            ON v.visit_concept_id = m_visit.concept_id 
    LEFT JOIN
        `concept` m_source_concept 
            ON measurement.measurement_source_concept_id = m_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
measurement_55136786_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "measurement_55136786",
  "measurement_55136786_*.csv")
message(str_glue('The data will be written to {measurement_55136786_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_55136786_measurement_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  measurement_55136786_path,
  destination_format = "CSV")



In [ ]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {measurement_55136786_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), measurement_type_concept_name = col_character(), operator_concept_name = col_character(), value_as_concept_name = col_character(), unit_concept_name = col_character(), visit_occurrence_concept_name = col_character(), measurement_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), unit_source_value = col_character(), value_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_55136786_measurement_df <- read_bq_export_from_workspace_bucket(measurement_55136786_path)

dim(dataset_55136786_measurement_df)

head(dataset_55136786_measurement_df, 5)

In [ ]:
sort(table(dataset_55136786_measurement_df$standard_concept_name))

In [ ]:
obs = dataset_55136786_measurement_df %>%
filter(standard_concept_name %in%
       c("Cholesterol in LDL [Mass/volume] in Serum or Plasma by calculation",
         "Cholesterol in LDL [Mass/volume] in Serum or Plasma",
         "Cholesterol in LDL [Mass/volume] in Serum or Plasma by Direct assay",
         "Cholesterol in LDL [Mass/volume] in Serum or Plasma ultracentrifugate",
         "Cholesterol in LDL [Mass/volume] in Serum or Plasma by Electrophoresis")) %>%
#filter(unit_concept_name %in%
#      c("milligram per deciliter", "No matching concept", "milligram per deciliter calculated", "milligram per milliliter", "mg/dL"))
select(person_id, measurement_datetime, value_as_number) %>%
na.omit() %>%
mutate(measurement_datetime = as.Date(measurement_datetime)) %>%
filter(value_as_number > 0) %>%
filter(value_as_number < 1000)

In [ ]:
x = quantile(obs$value_as_number, seq(0.001, 0.999, 0.001))
plot(x)
obs$value_as_number[(obs$value_as_number < min(x) | obs$value_as_number > max(x))] = NA

obs = obs[!is.na(obs$value_as_number), ]

In [ ]:
write.table(obs, "HDLLDL_srWGS.LDL.txt", row.names=FALSE, quote=FALSE, sep="\t")

In [ ]:
LDL190 = unique(x$person_id)
length(LDL190)

In [ ]:
x = dataset_55136786_measurement_df %>%
filter(standard_concept_name %in%
       c("Cholesterol in HDL [Mass/volume] in Serum or Plasma")) %>%
filter(unit_concept_name %in%
      c("milligram per deciliter")) %>%
select(person_id, value_as_number) %>%
na.omit() %>%
filter(value_as_number > 0) %>%
filter(value_as_number < 40)

In [ ]:
summary(x$value_as_number)

In [ ]:
HDL40 = unique(x$person_id)
length(HDL40)

There seems to be too many HDL40

In [ ]:
foo = read.table("lipidaemia_srWGS_withC10B_person.txt", header=TRUE, sep="\t")

In [ ]:
table(LDL190 %in% foo$person_id)

Most of LDL190 are already captured in lipidaemia_srWGS_withC10B_person.txt

In [ ]:
output =
dataset_55136786_measurement_df %>%
    filter(standard_concept_name == "Body mass index (BMI) [Ratio]") %>%
    select(person_id, value_as_number) %>%
    na.omit() %>%
    mutate(lower = quantile(value_as_number, 0.01),
           upper = quantile(value_as_number, 0.99)) %>%
    filter(value_as_number >= lower & value_as_number <= upper) %>%
    group_by(person_id) %>%
    summarize(BMI = median(value_as_number))
output

In [ ]:
write.table(
    output,
    sep = "\t",
    na = "NA",
    row.names = FALSE,
    quote = FALSE,
    file = "HDLLDL_srWGS_BMI.txt")